In [1]:
#!/usr/bin/env python

import os
import numpy as np
import pandas as pd

# ============================================================
# CONFIG
# ============================================================

# Momentum equity (baseline production run)
ROBUST_DIR    = "./13-trading_output_regression_insp500_spyfilter"
BASELINE_FILE = os.path.join(ROBUST_DIR, "13-equity_curve_regression_insp500_spyfilter.parquet")

# SPY regime file (your current schema)
SPY_REGIME_FILE = r"C:\TWS API\source\pythonclient\TradingIdeas\MomentumSystem\8-SPY_200DMA_market_regime\8-SPY_200DMA_regime.parquet"

TRADING_DAYS_PER_YEAR = 252

# ============================================================
# METRIC FUNCTIONS
# ============================================================

def cagr_from_curve(eq: pd.Series):
    eq = eq.dropna()
    if len(eq) < 2 or eq.iloc[0] <= 0:
        return np.nan
    years = (eq.index[-1] - eq.index[0]).days / 365.25
    if years <= 0:
        return np.nan
    return (eq.iloc[-1] / eq.iloc[0]) ** (1 / years) - 1


def max_drawdown(eq: pd.Series):
    eq = eq.dropna()
    if eq.empty:
        return np.nan
    peak = eq.cummax()
    return (eq / peak - 1.0).min()


def sharpe_ratio(eq: pd.Series):
    ret = eq.pct_change().dropna()
    if len(ret) < 2 or ret.std() == 0:
        return np.nan
    return ret.mean() / ret.std() * np.sqrt(TRADING_DAYS_PER_YEAR)


def sortino_ratio(eq: pd.Series):
    ret = eq.pct_change().dropna()
    downside = ret[ret < 0]
    if len(downside) == 0 or downside.std() == 0:
        return np.nan
    return ret.mean() / downside.std() * np.sqrt(TRADING_DAYS_PER_YEAR)


def total_return(eq: pd.Series):
    eq = eq.dropna()
    if len(eq) < 2:
        return np.nan
    return eq.iloc[-1] / eq.iloc[0] - 1.0


# ============================================================
# LOAD MOMENTUM EQUITY (CURRENT SCHEMA)
# ============================================================

eq_df = pd.read_parquet(BASELINE_FILE)

# Normalize date handling
if "date" in eq_df.columns:
    eq_df["date"] = pd.to_datetime(eq_df["date"])
    eq_df = eq_df.sort_values("date").set_index("date")
else:
    eq_df.index = pd.to_datetime(eq_df.index)
    eq_df = eq_df.sort_index()

if "portfolio_value" not in eq_df.columns:
    raise ValueError("Expected column 'portfolio_value' in equity file.")

eq_mom = eq_df["portfolio_value"].dropna()

print("Momentum equity loaded:")
print(eq_mom.index[0].date(), "→", eq_mom.index[-1].date())


# ============================================================
# LOAD SPY DATA (CURRENT REGIME SCHEMA)
# ============================================================

spy = pd.read_parquet(SPY_REGIME_FILE)

# Normalize date index
if "date" in spy.columns:
    spy["date"] = pd.to_datetime(spy["date"])
    spy = spy.set_index("date")
else:
    spy.index = pd.to_datetime(spy.index)

spy = spy.sort_index()

if "spy_close" not in spy.columns:
    raise ValueError("Expected 'spy_close' column in SPY regime file.")

spy_px = spy["spy_close"].dropna()

# Align date ranges
start = max(eq_mom.index.min(), spy_px.index.min())
end   = min(eq_mom.index.max(), spy_px.index.max())

eq_mom = eq_mom.loc[start:end]
spy_px = spy_px.loc[start:end]

# Normalize SPY to 1.0 at start
spy_eq = spy_px / spy_px.iloc[0]

print("SPY equity aligned:")
print(spy_eq.index[0].date(), "→", spy_eq.index[-1].date())


# ============================================================
# STRESS / CRISIS WINDOWS
# ============================================================

STRESS_PERIODS = {
    "DotCom_Bust":      ("2000-03-01", "2002-10-31"),
    "GFC_2008_2009":    ("2007-10-01", "2009-03-09"),
    "EuroCrisis_2011":  ("2011-04-01", "2011-10-31"),
    "COVID_Crash_2020": ("2020-02-15", "2020-03-31"),
    "Bear_2022":        ("2022-01-01", "2022-10-31"),
}


def clip(s: pd.Series, start, end):
    return s.loc[pd.to_datetime(start):pd.to_datetime(end)]


# ============================================================
# RUN STRESS ANALYSIS
# ============================================================

rows = []

for label, (start, end) in STRESS_PERIODS.items():
    sub_mom = clip(eq_mom, start, end)
    sub_spy = clip(spy_eq, start, end)

    if len(sub_mom) < 10 or len(sub_spy) < 10:
        print(f"Skipping {label}: insufficient data.")
        continue

    rows.append({
        "Period": label,
        "Start": sub_mom.index[0].date(),
        "End":   sub_mom.index[-1].date(),

        "Mom_TotalRet_%":  total_return(sub_mom) * 100,
        "Mom_CAGR_%":      cagr_from_curve(sub_mom) * 100,
        "Mom_MaxDD_%":     max_drawdown(sub_mom) * 100,
        "Mom_Sharpe":      sharpe_ratio(sub_mom),
        "Mom_Sortino":     sortino_ratio(sub_mom),

        "SPY_TotalRet_%":  total_return(sub_spy) * 100,
        "SPY_CAGR_%":      cagr_from_curve(sub_spy) * 100,
        "SPY_MaxDD_%":     max_drawdown(sub_spy) * 100,
        "SPY_Sharpe":      sharpe_ratio(sub_spy),
        "SPY_Sortino":     sortino_ratio(sub_spy),
    })


stress_df = pd.DataFrame(rows)

with pd.option_context("display.float_format", lambda x: f"{x:7.2f}"):
    print("\n=== STRESS PERIOD PERFORMANCE (Momentum vs SPY) ===")
    print(stress_df.to_string(index=False))


# ============================================================
# SAVE OUTPUT
# ============================================================

out_path = os.path.join(ROBUST_DIR, "stress_period_performance.csv")
stress_df.to_csv(out_path, index=False)

print(f"\nSaved stress period summary → {out_path}")


Momentum equity loaded:
1999-01-04 → 2025-12-12
SPY equity aligned:
1999-01-04 → 2025-12-11

=== STRESS PERIOD PERFORMANCE (Momentum vs SPY) ===
          Period      Start        End  Mom_TotalRet_%  Mom_CAGR_%  Mom_MaxDD_%  Mom_Sharpe  Mom_Sortino  SPY_TotalRet_%  SPY_CAGR_%  SPY_MaxDD_%  SPY_Sharpe  SPY_Sortino
     DotCom_Bust 2000-03-01 2002-10-31            1.31        0.49       -20.41        0.10         0.09          -33.85      -14.35       -47.52       -0.52        -0.86
   GFC_2008_2009 2007-10-01 2009-03-09          -14.42      -10.27       -18.83       -0.96        -0.78          -54.56      -42.23       -55.19       -1.26        -1.75
 EuroCrisis_2011 2011-04-01 2011-10-31           -5.79       -9.73       -12.65       -0.62        -0.78           -4.79       -8.07       -18.61       -0.20        -0.25
COVID_Crash_2020 2020-02-18 2020-03-31          -23.52      -90.28       -31.25       -3.59        -5.01          -23.00      -89.70       -33.72       -2.43        -3.99
